# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import time

from sklearn.tree import DecisionTreeClassifier, export_text, plot_tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.preprocessing import PolynomialFeatures, OneHotEncoder, OrdinalEncoder, LabelEncoder, StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.metrics import make_scorer, accuracy_score, recall_score, precision_score, mean_squared_error

from sklearn import set_config
set_config(display="diagram") #setting this will display your pipelines as seen above


In [7]:
# --- Data Source: ---- https://archive.ics.uci.edu/dataset/222/bank+marketing
# 
# Two available datasets: 
# 1) bank-additional-full.csv with all examples (41188) and 20 inputs, ordered by date (from May 2008 to November 2010), 
#    very close to the data analyzed in [Moro et al., 2014]
# 2) bank-additional.csv with 10% of the examples (4119), randomly selected from 1), and 20 inputs.
# 
# The smallest datasets provided to test more computationally demanding machine learning algorithms (e.g., SVM). 
# df = pd.read_csv('data/bank-additional/bank-additional-full.csv', sep = ';')

# Load "Dataset" from appropriate Directory
# df = pd.read_csv("/Users/m.jalloh/Downloads/module17_starter/mod17_data/bank-additional.csv", sep = ";")
# 
df_bank_term_deposit = pd.read_csv("/Users/m.jalloh/Downloads/module17_starter/mod17_data/bank-additional-full.csv", sep = ";")


In [8]:
df_bank_term_deposit.head()
df_bank_term_deposit.info()
df_bank_term_deposit.sample()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
39126,31,student,married,high.school,no,yes,no,cellular,dec,wed,...,3,999,0,nonexistent,-3.0,92.713,-33.0,0.7,5023.5,no


### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```



In [10]:
# check for missing data/ not-available data
# 
df_bank_term_deposit.isna()
df_bank_term_deposit.isna().sum().sum()


0

### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

In [12]:
# df.info()
df_bank_term_deposit.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

In [13]:
# Goal - Clearly state Business Objective of task
# 
# Objective -
# Predict whether a client will subscribe to a term deposit from a Portuguese Banking Institution.
# To accomplish this task, compare the performance of various classifiers (k-nearest neighbors, logistic regression, ...
# ... decision trees, support vector machines).
# using a dataset related to marketing campaign of Portuguese bank products based on phone calls.
# 


### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features, prepare the features and target column for modeling with appropriate encoding and transformations.

In [15]:
# prepare Features & Target columns
# 
X_features = df_bank_term_deposit.drop("y", axis = 1)
y_target = df_bank_term_deposit["y"]

# --- Encoding & Transformation ---

# separate numeric & categorical columns
# 
cat_cols = ["job", "marital", "default", "housing", "loan", "contact", "month", "day_of_week", "poutcome"]
num_cols = ["age", "duration", "campaign", "pdays", "previous", "emp.var.rate", "cons.price.idx", "cons.conf.idx", "euribor3m", "nr.employed"]
bank_cols = ["emp.var.rate", "cons.price.idx", "euribor3m"]

# use OneHotEncoder to transform non-numeric categories columns
# 
col_transformer = make_column_transformer(
    (OneHotEncoder(drop = 'if_binary'), cat_cols),
    remainder='passthrough',force_int_remainder_cols=False,
)

transformer = make_column_transformer(
    (OneHotEncoder(handle_unknown = "ignore"), cat_cols),
    (StandardScaler(), num_cols),
)

# -- Target, with LabelEncoder ---
labelEnc = LabelEncoder()
y_target_encoded = labelEnc.fit_transform(y_target)


In [16]:
col_transformer
# transformer

ColumnTransformer(force_int_remainder_cols=False, remainder='passthrough',
                  transformers=[('onehotencoder',
                                 OneHotEncoder(drop='if_binary'),
                                 ['job', 'marital', 'default', 'housing',
                                  'loan', 'contact', 'month', 'day_of_week',
                                  'poutcome'])])

### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

In [18]:
# Splitting Data into Train & Test Data
# 
X_train, X_test, y_train, y_test = train_test_split(X_features, y_target_encoded, test_size = 0.3, random_state = 22)


In [19]:
# X_train[["emp.var.rate"]]

### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

In [21]:
baseline_train = ''
baseline_test = ''
mse_baseline_train = ''
mse_baseline_test = ''

# YOUR CODE HERE
# split data to train & test data set
# 
X_train, X_test, y_train, y_test = train_test_split(X_features, y_target_encoded, test_size = 0.3, random_state = 22)

baseline_train_mean = y_train.mean()
baseline_train = np.full_like(y_train, baseline_train_mean)

baseline_test_mean = y_test.mean()
baseline_test = np.full_like(y_test, baseline_test_mean)

# -- performance of Baseline model via MMSE measure
# 
mse_baseline_train = mean_squared_error(baseline_train, y_train)
mse_baseline_test = mean_squared_error(baseline_test, y_test)


In [22]:
print(baseline_train_mean)

0.11140785959557421


In [23]:
print(mse_baseline_test)

0.11556202961883952


### Problem 8: A Simple Model

Use Logistic Regression to build a basic model on your data.  

In [25]:
# -- bank features to try: "emp.var.rate", "cons.price.idx", "euribor3m" ---
# 
X1 = X_train[["euribor3m"]]
LRmodel = LogisticRegression()

lr = LRmodel.fit(X1, y_train)

# -- performance of Simple model via MMSE measure
# 
model_1_train_mse = mean_squared_error(y_train, lr.predict(X1) )
model_1_test_mse =  mean_squared_error(y_test, lr.predict(X_test[["euribor3m"]]) )

print(model_1_train_mse)
print(model_1_test_mse)


0.11140785959557421
0.11556202961883952


### Problem 9: Score the Model

What is the accuracy of your model?

In [27]:
# -- bank features to try: "emp.var.rate", "cons.price.idx", "euribor3m" ---
# 
y_pred = lr.predict(X_test[["euribor3m"]])
print("accuracy = ", accuracy_score(y_test, y_pred))


accuracy =  0.8844379703811605


### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|     |    |.     |.     |

In [29]:
# -- Compare performance of Logistic Regresssion to other models ... KNN algorithm, Decision Tree, and SVM models
# 

X_train, X_test, y_train, y_test = train_test_split(X_features, y_target_encoded, test_size = 0.3, random_state = 22)

cat_cols = ["job", "marital", "default", "housing", "loan", "contact", "month", "day_of_week", "poutcome"]
num_cols = ["age", "duration", "campaign", "pdays", "previous", "emp.var.rate", "cons.price.idx", "cons.conf.idx", "euribor3m", "nr.employed"]
bank_cols = ["emp.var.rate", "cons.price.idx", "euribor3m"]

transformer = ""
models = ""

summary = {}

transformer = make_column_transformer(
    (OneHotEncoder(handle_unknown = "ignore"), cat_cols),
    (StandardScaler(), num_cols)
)
# -- models to explore --
models = {
    "LogReg" : LogisticRegression(),
    "knn" : KNeighborsClassifier(),
    "DTree" : DecisionTreeClassifier(),
    "SVM" : SVC()
}

#  --- Loop through various models using pipeline
# 
for label, classifier in models.items():
    pipe = Pipeline([
        ("transformer", transformer),
        ("model", classifier)
    ])
    
    # -- compute time to fit the models -- 
    start_train = time.time()
    pipe.fit(X_train, y_train)
    end_train = time.time()

    # -- record time to fit the models -- 
    train_time = end_train - start_train # time to fit model

    y_pred = pipe.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    # -- save summary of results --
    summary[label] = {
        "Train Time (sec)": train_time,
        "Accuracy": accuracy
    }
    
summaryDataFrame = pd.DataFrame.from_dict(summary, orient = "index").reset_index().rename(columns = {"index": "Model"})
print(summaryDataFrame)

    Model  Train Time (sec)  Accuracy
0  LogReg          0.089533  0.910091
1     knn          0.036028  0.898681
2   DTree          0.145537  0.888241
3     SVM          3.697192  0.909687


### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.


- Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree.  
- Adjust your performance metric

In [31]:
optimal_models = {}
scorer_type = make_scorer(accuracy_score)

# -- models to explore --
models = {
    "LogReg" : LogisticRegression(),
    "knn" : KNeighborsClassifier(),
    "DTree" : DecisionTreeClassifier(),
    "SVM" : SVC()
}

#--- setting up for hyperparameters Tuning & Grid Search ---

# -- define model parameters
search_space_grid = {
    "LogReg": {
        "models__C": [0.1, 1],
        "models__solver": ["liblinear"]
    },
    "knn": {
        "models__n_neighbors": [3, 7],
        "models__weights": ["uniform"]
    },
    "DTree": {
        "models__max_depth": [5, 9],
        "models__min_samples_split": [2]
    },  
    "SVM": {
        "models__C": [0.1, 1],
        "models__kernel": ["linear"],
        "models__gamma": ["scale"]
    }
}

# -- build a pipeline -----
# 
for label, classifier in models.items():
    pipe = Pipeline([
        ("transformer", transformer),
        ("models", classifier)
    ])    
    grid_search = GridSearchCV(
        estimator = pipe,
        param_grid = search_space_grid[label],
        scoring = scorer_type,
        cv = 5
    )
    
    # --- perform Grid Search for models within pipeline ---
    grid_search.fit(X_train, y_train)
    optimal_models[label] = {
        "optimal_estimator": grid_search.best_estimator_,
        "optimal_parameters": grid_search.best_params_,
        "optimal_score_": grid_search.best_score_
    }

# --- display Summary of results ---
for models, info in optimal_models.items():
    print(models, info["optimal_parameters"], info["optimal_score_"])



LogReg {'models__C': 1, 'models__solver': 'liblinear'} 0.9110334007146885
knn {'models__n_neighbors': 7, 'models__weights': 'uniform'} 0.9042699077080529
DTree {'models__max_depth': 5, 'models__min_samples_split': 2} 0.912212547367084
SVM {'models__C': 1, 'models__gamma': 'scale', 'models__kernel': 'linear'} 0.9042351795151056


##### Questions